# Advent of Code 2022

## Day 11: Monkey in the Middle

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Today's part 2 required a bit of maths rather than just blind coding! I think that's the first time in this year's AoC that needed what I'd describe as mathematical thinking. But not too difficult, so I still managed to solve it myself.

Firstly, in part 1, I build a `Monkey` class and a generator which loops over the file yielding `Monkey` objects to load the data. I used this as mutable state to track the state of the monkeys. The list of items is a `deque` although I'm sure a `list` would have worked fine. There's a lot of printing code in my logic behind this `verbos=` flag that gets passed to methods fo I could compare against the sample output in the question.

I added a progress bar with `tqdm` to see how long part 2 was going to take without modification to the code other than removing the divide-by-3 and setting the rounds to 10,000. It would take too long. There item worry numbers are too big to compute even with Python's unbounded ints.

I modified the method which applies the operator to divide by the product of all monkey divisors. They are all primes. I assume this is the lowest base I can use inless there's some other trick I'm missing? This works first time!

Very fun.

### Imports

In [ ]:
from typing import Iterator, Union, Optional
from collections import deque

import tqdm.notebook as tqdm

### Data File Processing

In [ ]:
def read_data_blocks(filename: str) -> Iterator[list[str]]:

    buffer = []

    with open(filename) as file:

        for line in file:

            line = line.strip('\n')
            if line:
                buffer.append(line)
            else:
                yield buffer
                buffer = []

    # in case the final block is not terminated with \n
    if buffer:
        yield buffer

In [ ]:
def strip_line(line: str, start: str, end: str='') -> str:
    assert line.startswith(start), f'line "{line}" does not start with "{start}"'
    assert line.endswith(end), f'line "{line}" does not end with "{end}"'
    if end:
        return line[len(start):-len(end)]
    return line[len(start):]

### Monkey Object

In [ ]:
class Monkey:

    __slots__ = ['number', 'items', 'operator', 'operand', 'divisor', 'if_true', 'if_false', 'inspections']

    def __init__(self,
                 number: int,
                 items: list[int],
                 operator: str,
                 operand: Union[str, int],
                 divisor: int,
                 if_true: int,
                 if_false: int):
        self.number = number
        self.items = deque(items)
        self.operator = operator
        self.operand = operand
        self.divisor = divisor
        self.if_true = if_true
        self.if_false = if_false
        self.inspections = 0

    def inspect(self, item: int, verbose: bool=False) -> None:
        self.inspections += 1
        if verbose:
            print(f'  Monkey inspects an item with a worry level of {item}.')

    def apply_operator(self, old: int, verbose: bool=False, N:Optional[int]=None) -> int:

        if self.operator == '*':

            multiplier = old if self.operand == 'old' else self.operand
            new = multiplier * old
            if verbose:
                print(f'    Worry level is multiplied by {multiplier} to {new}.')

        else:

            delta = old if self.operand == 'old' else self.operand
            new = delta + old
            if verbose:
                print(f'    Worry level increases by {delta} to {new}.')

        if N:
            return new % N
        return new

    def __str__(self):
        return f'Monkey {self.number}: {", ".join([str(i) for i in self.items])}'

    def __repr__(self):
        return str(self)

In [ ]:
def read_monkeys(filename: str) -> Iterator[Monkey]:

    for block in read_data_blocks(filename):

        assert len(block) == 6
        monkey_number, starting, op, test, if_true, if_false = block

        monkey_number = int(strip_line(monkey_number, 'Monkey ', ':'))
        starting = strip_line(starting, '  Starting items: ')
        starting = [int(e) for e in starting.split(', ')]
        op = strip_line(op, '  Operation: new = old ')
        operator, operand = op.split(' ')
        try:
            operand = int(operand)
        except ValueError:
            pass
        test = int(strip_line(test, '  Test: divisible by '))
        if_true = int(strip_line(if_true, '    If true: throw to monkey '))
        if_false = int(strip_line(if_false, '    If false: throw to monkey '))

        yield Monkey(monkey_number, starting, operator, operand, test, if_true, if_false)

### Processing Rounds of Monkey Business

In [ ]:
def monkey_round(monkeys: list[Monkey], worry_reduction: bool=True, verbose:bool=False, N:Optional[int]=None) -> None:

    for monkey in monkeys:

        if verbose:
            print(f'Monkey {monkey.number}:')

        while monkey.items:
            item = monkey.items.popleft()
            monkey.inspect(item, verbose=verbose)
            item = monkey.apply_operator(item, verbose=verbose, N=N)

            if worry_reduction:
                item = item // 3
                if verbose:
                    print(f'    Monkey gets bored with item. Worry level is divided by 3 to {item}.')

            if item % monkey.divisor == 0:
                if verbose:
                    print(f'    Current worry level is divisible by {monkey.divisor}')
                destination = monkey.if_true
            else:
                if verbose:
                    print(f'    Current worry level is not divisible by {monkey.divisor}')
                destination = monkey.if_false
            if verbose:
                print(f'    Item with worry level {item} is thrown to monkey {destination}')
            monkeys[destination].items.append(item)

In [ ]:
def print_state(monkeys: list[Monkey], rounds_complete: int) -> None:
    print(f'After {rounds_complete} rounds...')
    for monkey in monkeys:
        print(monkey)
    print()

In [ ]:
def do_all_rounds(monkeys: list[Monkey],
                  num_rounds: int,
                  worry_reduction: bool=True,
                  verbose: bool=False,
                  progress_bar: bool=False,
                  N:Optional[int]=None) -> int:

    if verbose:
        print_state(monkeys, 0)

    iterator = range(num_rounds)
    if progress_bar:
        iterator = tqdm.tqdm(iterator)

    for rounds_complete in iterator:
        monkey_round(monkeys, worry_reduction=worry_reduction, verbose=verbose, N=N)
        if verbose:
            print_state(monkeys, rounds_complete+1)

    inspections = []
    for monkey in monkeys:
        if verbose:
            print(f'Monkey {monkey.number} inspected items {monkey.inspections} times.')
        inspections.append(monkey.inspections)
    inspections.sort(reverse=True)
    monkey_business = inspections[0] * inspections[1]
    if verbose:
        print()

    return monkey_business

### Part 1

In [ ]:
INPUT_FILE = 'data/input11.txt'

In [ ]:
def main(num_rounds: int, worry_reduction: bool, verbose: bool, progress_bar: bool, N:Optional[int]=None) -> None:

    monkeys = []

    for monkey in read_monkeys(INPUT_FILE):
        assert monkey.number == len(monkeys)
        monkeys.append(monkey)

    monkey_business = do_all_rounds(monkeys,
                                    num_rounds,
                                    worry_reduction= worry_reduction,
                                    verbose=verbose,
                                    progress_bar=progress_bar,
                                    N=N)

    print(f'The level of monkey business after {num_rounds} rounds is {monkey_business}')

In [ ]:
if __name__ == '__main__':
    main(num_rounds=20, worry_reduction=True, verbose=False, progress_bar=False, N=None)

### Testing Stuff

Applying the same code to part 2 without `worry_reduction` looks like it will take forever to finish due to the huge numbers.

There needs ot me a maths trick.

What's the product of the divisors of the tests?

Maybe we can perform arithmetic mod this?

In [ ]:
def check_product_of_divisors():

    monkeys = []

    for monkey in read_monkeys(INPUT_FILE):
        assert monkey.number == len(monkeys)
        monkeys.append(monkey)

    N = 1
    for m in monkeys:
        N *= m.divisor

    return N

print('product of divisors =', check_product_of_divisors())

### Part 2

I've gone back and edited the `Monkey.apply_operator` to accept the argument `N` and divide by this number and take the remainder is it's not set to `None`.

In [ ]:
if __name__ == '__main__':
    N = check_product_of_divisors()
    main(num_rounds=10000, worry_reduction=False, verbose=False, progress_bar=True, N=N)

Success.